### Real World application 6 (Chapitre 4)
create database stack 

use stack

In [5]:
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship
import pandas as pd

# Connecter à la base de données Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="stack")

# Charger l'ensemble de données Stack
stack_network_nodest_df = pd.read_csv('stack_network_nodes.csv')
stack_network_links_df = pd.read_csv('stack_network_links.csv')

# Créer les nœuds correspondants dans la base de données Neo4j
for index, row in stack_network_nodest_df.iterrows():
    node_name = row['name']
    node_group = row['group']
    node_size = row['nodesize']
    node = Node("Keyword", name=node_name, group=node_group, nodesize=node_size)
    graph.create(node)

# Créer les relations entre les nœuds dans la base de données Neo4j
for index, row in stack_network_links_df.iterrows():
    source_name = row['source']
    target_name = row['target']
    weight = row['value']
    
    source_node = graph.nodes.match("Keyword", name=source_name).first()
    target_node = graph.nodes.match("Keyword", name=target_name).first()
    
    if source_node and target_node:
        relation = Relationship(source_node, "CO_OCCURRENCE_WITH", target_node, weight=weight)
        graph.create(relation)

        
# 1) What are the 10 most popular keywords?
query_1 = """
MATCH (n)
RETURN n.name AS keyword, n.nodesize AS popularity
ORDER BY popularity DESC
LIMIT 10
"""

result_1 = graph.run(query_1).to_data_frame()
print("Top 10 most popular keywords:")
print(result_1)

Top 10 most popular keywords:
      keyword  popularity
0  javascript      649.16
1        java      610.65
2      python      438.67
3         php      361.22
4         css      341.17
5          c#      321.13
6        html      272.45
7         c++      268.11
8     android      229.86
9      jquery      208.29


In [3]:
print(stack_network_nodest_df.head())

        name  group  nodesize
0       html      6    272.45
1        css      6    341.17
2  hibernate      8     29.83
3     spring      8     52.84
4       ruby      3     70.14


In [4]:
print(stack_network_links_df.head())

             source target      value
0             azure   .net  20.933192
1        sql-server   .net  32.322524
2           asp.net   .net  48.407030
3  entity-framework   .net  24.370903
4               wpf   .net  32.350925


In [6]:
from py2neo import Graph

# Connect to Neo4j database
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="stack")


# 2) What are the top 10 Co-occurrences?
query_2 = """
MATCH (source)-[link]->(target)
RETURN source.name AS source_keyword, target.name AS target_keyword, link.value AS co_occurrence
ORDER BY co_occurrence DESC
LIMIT 10
"""

result_2 = graph.run(query_2).to_data_frame()
print("\nTop 10 co-occurrences:")
print(result_2)


Top 10 co-occurrences:
     source_keyword target_keyword co_occurrence
0        sql-server           .net          None
1           asp.net           .net          None
2  entity-framework           .net          None
3               wpf           .net          None
4              linq           .net          None
5               wcf           .net          None
6                c#           .net          None
7               tdd          agile          None
8       codeigniter           ajax          None
9             azure           .net          None


In [7]:
from py2neo import Graph

# Connect to Neo4j database
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="stack")

# 3) How many keywords are in group 1?
query_3 = """
MATCH (n)
WHERE n.group = 1
RETURN count(n) AS num_keywords_in_group_1
"""

result_3 = graph.run(query_3).to_data_frame()
print("\nNumber of keywords in group 1:")
print(result_3)


Number of keywords in group 1:
   num_keywords_in_group_1
0                       10


In [8]:
from py2neo import Graph

# Connect to Neo4j database
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="stack")

# 4) Is the Co-occurrence correlated to the "group" of the keyword?
query_4 = """
MATCH (source)-[link]->(target)
RETURN source.group AS keyword_group, avg(link.value) AS avg_co_occurrence
ORDER BY keyword_group
"""

result_4 = graph.run(query_4).to_data_frame()
print("\nAverage co-occurrence per keyword group:")
print(result_4)


Average co-occurrence per keyword group:
    keyword_group avg_co_occurrence
0               1              None
1               2              None
2               3              None
3               4              None
4               5              None
5               6              None
6               7              None
7               8              None
8               9              None
9              10              None
10             11              None
11             12              None
12             13              None
13             14              None
